# Data Exploration
----

This notebook explores and accesses data from the file based database for buffalo trace

In [1]:
import sqlalchemy 
import pandas as pd 
from importlib import resources
import datetime as dt
from plotly import express as pe

In [2]:
with resources.path("bourbon","buffalo_trace.db") as f: 
    my_conn = sqlalchemy.create_engine(f"sqlite:////{f.absolute()}")

latest_date = my_conn.execute("select max(update_date) from product_avail").fetchone()[0]
today_batch = pd.read_sql(f"select distinct * from product_avail where update_date='{latest_date}'", con=my_conn)
today_batch

/var/folders/vz/x1_phppx4jlfxt612qb8vcs80000gn/T/ipykernel_97989/3650293179.py:4: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  latest_date = my_conn.execute("select max(update_date) from product_avail").fetchone()[0]


,product_title,update_date,product_available
0,Buffalo Trace,2023-01-16 12:35:00.000000,0
1,Eagle Rare,2023-01-16 12:35:00.000000,0
2,E.H. Taylor Jr. Small Batch,2023-01-16 12:35:00.000000,0
3,Blanton’s Single Barrel,2023-01-16 12:35:00.000000,0
4,W.L. Weller Special Reserve,2023-01-16 12:35:00.000000,0
5,Sazerac Rye,2023-01-16 12:35:00.000000,1
6,Wheatley,2023-01-16 12:35:00.000000,1
7,Bourbon Cream,2023-01-16 12:35:00.000000,0


## Clean SQL

In [20]:
pd.read_sql("select distinct update_date, count(product_title) as product_count from product_avail group by update_date", con=my_conn)

,update_date,product_count
0,2023-01-16 10:00:00.000000,8
1,2023-01-16 10:45:00.000000,48


In [17]:
pd.read_sql("select * from product_ext", con=my_conn)

,product_title,product_image
0,Buffalo Trace,http://buffalotracedistillery.com/content/dam/...
1,Eagle Rare,http://buffalotracedistillery.com/content/dam/...
2,E.H. Taylor Jr. Small Batch,http://buffalotracedistillery.com/content/dam/...
3,Blanton’s Single Barrel,http://buffalotracedistillery.com/content/dam/...
4,W.L. Weller Special Reserve,http://buffalotracedistillery.com/content/dam/...
5,Sazerac Rye,http://buffalotracedistillery.com/content/dam/...
6,Wheatley,http://buffalotracedistillery.com/content/dam/...
7,Bourbon Cream,http://buffalotracedistillery.com/content/dam/...


In [19]:
dt.datetime.strptime(my_conn.execute("select max(update_date) from product_avail").fetchone()[0], '%Y-%m-%d %H:%M:%S.%f')

datetime.datetime(2023, 1, 16, 10, 45)